
 # Valorant Masters Toronto 2025 Player Statistics Scraper
 This script scrapes player statistics from the Valorant Masters Toronto 2025 event page on vlr.gg, processes the data, and saves it to a CSV file.

In [11]:
import urllib.request  # For fetching the webpage content
from bs4 import BeautifulSoup  # For parsing HTML content
import csv
import os

In [12]:
# Helper functions
def clean_text(text):
    """Remove unwanted characters and extra whitespace from text."""
    return ' '.join(text.strip().split())

In [13]:
vct_toronto_2025_url = "https://www.vlr.gg/event/stats/2282/valorant-masters-toronto-2025?exclude=32232&min_rounds=0&agent=all"
main_table_classname = "wf-table mod-stats mod-scroll"

In [15]:
contents = urllib.request.urlopen(vct_toronto_2025_url).read()
soup = BeautifulSoup(contents, "html.parser")
stats_table = soup.find("table", {"class": main_table_classname})
rows = stats_table.find_all('tr')

In [5]:
# Data Matrix
players_stat_mat = []
headers_raw = rows[0]
headers = headers_raw.find_all('th')
header_list = [clean_text(header.text) for header in headers]  # Clean header text
# Insert 'Team' column header after 'Player'
header_list.insert(1, 'Team')
players_stat_mat.append(header_list)

In [6]:
for idx, row in enumerate(rows[1:]):
    # if idx != 0:
        # continue
    tds = row.find_all("td")  # Get all table cells in the row
    data = []
    for td in tds:
        if "class" in td.attrs:
            # Handle player name and team
            if "mod-player" in td["class"]:
                name_and_team = td.get_text(",", True).split(",")
                for ele in name_and_team:
                    data.append(clean_text(ele))
                continue

            # Handle agent icons
            elif "mod-agents" in td["class"]:
                algs = td.find_all("img")
                agnts = [a["src"].split("/")[-1][:-4] for a in algs]  # Extract agent names from image URLs
                data.append(agnts)  # Append agent list
                continue
        
        # Handle other statistics
        data.append(clean_text(td.text))
    
    players_stat_mat.append(data)  # Append processed row to matrix

In [7]:
parent_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(parent_dir, 'dataset', 'vct_toronto_2025', 'player_stats.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write to CSV
with open(file_path, 'a+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(players_stat_mat)

In [10]:
print(f'Total Rows affected: {len(players_stat_mat)-1}' )

Total Rows affected: 61
